In [46]:
## NEXT STEPS:
##      TAKE OUT SPECIAL CHARACTERS
##      CONVERT TEXT INTO LOWERCASE
##      REMOVE STOPWORDS AND PUNCTUATION
##      ADD LABEL
##      READ ALL FILES FROM FOLDER AND COMBINE THEM INTO A SINGLE CSV
##      READ FILES FROM S3
##      CONVERT TO GLUE CODE

### EXTRA
##      TOKENIZATION
##      STEMMING/LEMMATIZATION

import os
import re
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, regexp_replace, lit, size, udf
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType, MapType

# Initialize Spark session
spark = SparkSession.builder \
    .appName("HotelDataProcessing") \
    .getOrCreate()

# Path to the local JSON file
json_file_path = '../keys/dummy_2024-07-10.json'  # Update this path
output_path = '../output/'  # Update this path as needed

# Extract region from the file name
file_name = os.path.basename(json_file_path)
region = file_name.split('_')[0]

# Define the schema
schema = StructType([
    StructField("hotel_name", StringType(), True),
    StructField("location", StringType(), True),
    StructField("rating", StringType(), True),
    StructField("user_ratings_total", IntegerType(), True),
    StructField("max_number_of_people", IntegerType(), True),
    StructField("address", StringType(), True),
    StructField("business_status", StringType(), True),
    StructField("place_id", StringType(), True),
    StructField("amenities", MapType(StringType(), StringType()), True),
    StructField("photos", ArrayType(
        StructType([
            StructField("photo_reference", StringType(), True),
            StructField("s3_url", StringType(), True),
            StructField("html_attributions", ArrayType(StringType()), True)
        ])
    ), True),
    StructField("reviews", ArrayType(
        StructType([
            StructField("user", StringType(), True),
            StructField("rating", StringType(), True),
            StructField("date", StringType(), True),
            StructField("review", StringType(), True)
        ])
    ), True),
    StructField("source", StringType(), True)
])

# Load JSON data from the local file with schema and multiline option
df = spark.read.schema(schema).option("multiline", "true").json(json_file_path)

# Print the schema and contents of the DataFrame
print("Schema of the DataFrame:")
df.printSchema()
print("Contents of the DataFrame:")
df.show(truncate=False)

# Explode the reviews array to analyze individual reviews
exploded_df = df.select(
    col("hotel_name"),
    col("rating"),
    col("user_ratings_total"),
    col("address"),
    col("business_status"),
    size(col("photos")).alias("number_of_photos"),  # Add column for number of photos
    explode(col("reviews")).alias("review")
)

# Define a UDF to convert relative dates to number of days
def convert_to_days(date_str):
    try:
        if "day" in date_str:
            days = int(re.findall(r'\d+', date_str)[0])
        elif "week" in date_str:
            days = int(re.findall(r'\d+', date_str)[0]) * 7
        elif "month" in date_str:
            days = int(re.findall(r'\d+', date_str)[0]) * 30
        elif "year" in date_str:
            days = int(re.findall(r'\d+', date_str)[0]) * 365
        else:
            days = 0
    except (ValueError, IndexError):
        days = 0
    return days

convert_to_days_udf = udf(convert_to_days, IntegerType())

# Flatten the DataFrame, replace newline characters with spaces in reviews, and convert dates
flattened_df = exploded_df.select(
    lit(region).alias("region"),  # Add region as a new column
    col("hotel_name"),
    col("rating"),
    col("user_ratings_total"),
    col("number_of_photos"),  # Include the number of photos column
    col("address"),
    col("business_status"),
    col("review.user").alias("review_user"),
    col("review.rating").alias("review_rating"),
    convert_to_days_udf(col("review.date")).alias("review_date_in_days"),  # Convert review date to days
    regexp_replace(col("review.review"), "\n", " ").alias("review_text")
)

# Show the transformed data
print("Transformed DataFrame:")
flattened_df.show(truncate=False)


# Save the transformed data to CSV with the region and number_of_photos columns included
flattened_df.write.mode("overwrite").csv(output_path, header=True)

# Stop the Spark session
spark.stop()

Schema of the DataFrame:
root
 |-- hotel_name: string (nullable = true)
 |-- location: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- user_ratings_total: integer (nullable = true)
 |-- max_number_of_people: integer (nullable = true)
 |-- address: string (nullable = true)
 |-- business_status: string (nullable = true)
 |-- place_id: string (nullable = true)
 |-- amenities: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- photos: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- photo_reference: string (nullable = true)
 |    |    |-- s3_url: string (nullable = true)
 |    |    |-- html_attributions: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |-- reviews: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- user: string (nullable = true)
 |    |    |-- rating: string (nullable = true)
 |    |    |-- date: st